### Designing multiple finite state automata using inheritance in Python

CS 236 <br>
Fall 2023

Michael A. Goodrich <br>
Brigham Young University <br>
Fall 2023
***

Project 1 requires you to create a set of FSAs, one for each legal token type allowed in the _Datalog_ database language. Here are the first several token types from the project.

| Token Type    | Description           |  
| :--           | :--                   |  
| COMMA         | The ',' character     |
| PERIOD        | The '.' character     |
| Q_MARK        | The '?' character     |
| LEFT_PAREN    | The '(' character     |
| RIGHT_PAREN   | The ')' character     |
| COLON         | The ':' character     |
| COLON-DASH    | The string ":-"       |

Since we have to build a lot of FSAs, we'll use good software engineering practice and use inheritance.

***

We'll use the inheritance pattern from 

https://www.geeksforgeeks.org/inheritance-in-python/

which says the following:

"If you forget to invoke the __init__() of the parent class then its instance variables would not be available to the child [derived] class." 

The error produced by the inheritance patter will look like:

    Traceback (most recent call last):
    File "myfile.py", line 12, in 
        method_name
    AttributeError: 'Derived_class' object has no attribute 'method_name'



***
Let's approach this by copying the essential pieces of the FSA class from the _FSA_code_example_ notebook. Observe the following differences from the _FSA_code_example_ notebook:

 * I called the base class simply __FSA__ rather than __colonDash_FSA__
 * I passed the name of the FSA in as an argument to the constructor
 * The constructor leaves the set of accept states undefined.
 * The constructor also leaves the FSA name empty
 * The starting state is S0, but the body of the S0 method only raises an error.
 * No methods are defined for any other state.
 
I also removed the part of the method that tracked history since we won't use it in this tutorial.

Go ahead and execute the code below. That will define the base class, and we can "inherit from it" in later parts of the tutorial.

In [3]:
class FSA:
    """ FSA Base or Super class"""
    def __init__(self,name):
        """ Class constructor """
        ###############################################################
        # Define the five elements of the FSA
        ###############################################################
        # Set of states: Each state S0, S1, S2, Serr will be represented by its own method
        # Set of inputs: I is the set of alphanumeric characters, checked by isalnum()
        self.start_state = self.S0  # We'll always have the start state named S0
        self.accept_states = set()  # No accept states defined
        # Transition function: Each transition will be defined in the state methods
        
        ###############################################################
        # Define four variables that are used within the FSA, some
        # to make the FSA run and some that help us understand how 
        # the FSA works
        ###############################################################
        self.input_string = ""      # Default empty input
        self.FSA_name = name
        self.num_chars_read = 0
            
    #########################################################
    # Define each state as a function                       #
    # Within each function, define the transition function. #
    # Each transition function reads the current input and  #
    # choose the next state based on the input              #
    #########################################################
    def S0(self):
        """ Every FSA must have a start state, and we'll always name 
        it S0. The method for the start state must be defined in the
        derived class since it's not defined here. """
        raise NotImplementedError()     # This line causes an error to 
                                        # occur if the child classes don't implement this method

    ############################
    # Public Manager Functions
    ############################
    def Run(self,input_string):
        ###############################################################
        # This function will be called to make the FSA execute
        # It records the input string,
        # sets the current state to the start state
        # and then calls a state method that returns the next state
        # Each state method accesses the input_string
        ###############################################################
            # Remember input_string
        self.input_string = input_string
            # Set current state to start state
        current_state = self.start_state
            # Call current state, which starts the FSA
        while self.num_chars_read < len(self.input_string):
            current_state = current_state()
            # Check whether the FSA ended in an accept state
        outcome = False 
        if current_state in self.accept_states: outcome = True # Accept if the FSA ended in an accept state
        return outcome
    def Reset(self):
        self.num_chars_read = 0
        self.history = []
    
    ############################
    # Public Getters and Setters
    ############################
    def get_Name(self): return self.FSA_name
    def set_Name(self,FSA_name): self.FSA_name = FSA_name
    
    ############################
    # Private Helper functions
    ############################
    def __getCurrentInput(self):  # The double underscore makes the method private
        current_input = self.input_string[self.num_chars_read]
        self.num_chars_read += 1
        return current_input


***
We can create a derived (or child) class from the FSA base class by doing three things. First, we include the name of the base class inside the parentheses of the derived class:

        class derived_class(base_class):

Second, we invoke the constructor of the base class from within the constructor of the derived class, which must be done or bad stuff happens:

        def __init__(self,name):
                base_class.__init__(self,name)
                # Rest of constructor

Third, you must define a method in the child_class for each unimplemented method in the parent_class. In our case, the unimplemented method in the parent class is __S0__.

Once these three things are done, you can add whatever other methods you want to the derived_class. I'll add the states that make the derived_class perform like the colonDash_FSA from the _FSA_code_example_ Jupyter notebook.

***
The code example that follows calls the constructor, which calls the construct of the base_class and defines an accept state. The code then defines the methods for each state in the colonDash finite state automaton. I just copied these methods straight from the _FSA_code_example_ Jupyter notebook (except I dropped the history tracking that we used to construct the trace).

In [4]:
class colonDash_FSA(FSA):
    def __init__(self):
        FSA.__init__(self,"colonDash_FSA") # You must invoke the __init__ of the parent class
        self.accept_states.add(self.S2) # Since self.accept_states is defined in parent class, I can use it here
    
    def S0(self):
        current_input = self.__getCurrentInput()
        if current_input == ':': next_state = self.S1
        else: next_state = self.Serr
        return next_state
    def S1(self):
        current_input = self.__getCurrentInput()
        if current_input == '-': next_state = self.S2
        else: next_state = self.Serr
        return next_state
    def S2(self):
        current_input = self.__getCurrentInput()
        next_state = self.S2 # loop in state S2
        return next_state
    def Serr(self):
        current_input = self.__getCurrentInput()
        next_state = self.Serr # loop in state Serr
        return next_state
    

I love how clean this code is. Let's run it and see if it works.

In [ ]:
my_FSA = colonDash_FSA()
input_string = ":-"
accept_status = my_FSA.Run(input_string)
if accept_status: print("The ", my_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")
